In [1]:
import torch
from dagma import utils
from dagma.linear import DagmaLinear
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
import pandas as pd
import networkx as nx
from pywhy_graphs.viz import draw
import numpy as np

/Users/vv/project-OperatorsOfTheNeuron/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from causallearn.graph.GraphNode import GraphNode
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from causallearn.search.ConstraintBased.PC import pc
from networkx import DiGraph

In [3]:
utils.set_random_seed(1)

In [4]:
model = DagmaLinear(loss_type='l2')

In [5]:
gt = np.loadtxt('gt.txt', delimiter=' ')

FileNotFoundError: gt.txt not found.

In [83]:
d1 = pd.read_csv("dataset/low_scrap.csv")
d2 = pd.read_csv("dataset/high_scrap.csv")
data = pd.concat([d1, d2], axis=0)

stations = data.keys()
stations = [station[7:] for station in stations]
data.columns = stations
station_pairs = [(station.split("_")[0], station.split("_")[2]) for station in stations]

# for col in data.columns:
#     if np.array_equal(data[col].values, data[col].values.astype(int)):
#         continue
#     # normalize
#     data[col] = (data[col] - data[col].mean()) / data[col].mean()

# d = Discretiser(method="quantile", num_buckets=10)
# discrete_data = d.fit_transform(data.to_numpy())

# data = pd.DataFrame(discrete_data, columns=data.columns)

In [84]:
excluded_edges = []
for i, s in enumerate(station_pairs):
    for j, t in enumerate(station_pairs):
        if t[0] < s[0]:
            # excluded_edges.append((str(t[0]) + "_mp_" + str(t[1]), str(s[0]) + "_mp_" + str(s[1])))
            excluded_edges.append((i, j))

In [85]:
ee_tuple = tuple(excluded_edges)

In [107]:
# for T in range(8, 12, 2):
#     for w_thresh in [0.3, 0.35, 0.4, 0.45, 0.5, 0.55]:
A = model.fit(data.to_numpy(), exclude_edges=ee_tuple, T=10, w_threshold=0.3)

  0%|          | 0/330000.0 [00:00<?, ?it/s]

In [108]:
# create networkx graph
import networkx as nx
import matplotlib.pyplot as plt

threshold = 0.0

G = nx.DiGraph()
G.add_nodes_from(range(len(A)))
for i in range(len(A)):
    for j in range(len(A)):
        if abs(A[i][j]) > threshold:
            G.add_edge(i, j, weight=A[i, j])

In [109]:
nodes = [GraphNode(station) for station in stations]
# dot_graph = draw(graph, name="DAG with (z, y) directed edge")
# dot_graph.render(outfile=f"red.png", format="png", cleanup=True)

bk = BackgroundKnowledge()
for s in station_pairs:
    bk.add_node_to_tier(nodes[int(s[1])], int(s[0]))

for edge in G.edges():
    # get indices of the nodes
    bk.add_required_by_node(nodes[edge[0]], nodes[edge[1]])

In [110]:
cg = pc(data.to_numpy(), alpha=0.125, background_knowledge=bk, )

  0%|          | 0/98 [00:00<?, ?it/s]

In [111]:
Gp = DiGraph()
Gp.add_nodes_from(stations)
A = np.array(cg.G.graph)
for i in range(len(stations)):
    for j in range(len(stations)):
        if cg.G.graph[i, j] == 1 and not (i,j) in excluded_edges:
            Gp.add_edge(stations[i], stations[j])

In [112]:
dot_graph = draw(Gp, name="DAG with (z, y) directed edge")
dot_graph.render(outfile=f"composite.png", format="png", cleanup=True)

'composite.png'

In [113]:
comp = nx.to_numpy_array(Gp)
np.savetxt("composite.csv", comp.astype(int), delimiter=",")